在使用逻辑回归、决策树等模型算法构建分类模型时，经常需要对特征变量进行筛选。因为有时可能会获得100多个候选特征变量，通常不会直接把这些特征变量放到模型中去进行拟合训练，而是从这些特征变量中挑选一些放进模型，构成入模变量列表。那么该如何挑选入模变量呢？

挑选入模变量需要考虑很多因素，如变量的预测能力、简单性（容易生成和使用）、可解释性等。其中最主要的衡量标准是变量的预测能力，对分类模型来说，即希望变量具有较好的特征区分度，可以较准确地将样本进行分类。

WOE值和IV值就是这样的指标，它们可以用来衡量特征变量的预测能力，或者说特征变量的特征区分度，类似的指标还有前面讲到的基尼系数和信息增益。对于决策树等树模型来说，可以通过基尼系数或信息增益来衡量变量的特征区分度，而对于逻辑回归等没有基尼系数等指标的模型而言，可以通过WOE值和IV值进行变量选择。IV值的计算是以WOE值为基础的，而要计算一个变量的WOE值，需要对这个变量进行分箱处理。

WOE值与IV值在银行的信贷违约预测中应用较多，如利用逻辑回归模型搭建违约预测模型。

## 一、WOE值
WOE是Weight of Evidence（证据权重）的缩写，其反映了某一变量的特征区分度。各个分箱的WOE值（更准确地说是WOE值的绝对值）都很大，说明各个分箱的区分度很高，即能够较好地进行分类，从这个角度来说，WOE值类似于决策树的基尼系数和信息增益。

## 二、IV值
IV是Information Value（信息量）的缩写。在进行特征筛选时，IV值能较好地反映特征变量的预测能力，特征变量对预测结果的贡献越大，其价值就越大，对应的IV值就越大，因此，我们可根据IV值的大小筛选出需要的特征变量。
> 一个特征变量的IV值越高，说明该特征变量越具有区分度。不过IV值也不是越大越好，如果一个特征变量的IV值大于0.5，有时需要对这个特征变量持有疑问，因为它有点过好而显得不够真实。通常会选择IV值在0.1～0.5这个范围内的特征变量。

## 示例：客户流失预警模型的IV值计算

In [10]:
import pandas as pd
import numpy as np

def cal_iv(data, cut_num, feature, target):
    # 1、数据分箱
    data_cut = pd.cut(data[feature], cut_num)
    
    # 2、统计各个分箱的总样本数、坏样本数和好样本数
    cut_group_all = data[target].groupby(data_cut).count() # 总样本数
    cut_y = data[target].groupby(data_cut).sum() # 坏样本数
    cut_n = cut_group_all - cut_y # 好样本数
    # 汇总基础数据
    df = pd.DataFrame() # 创建一个空DataFrame来汇总数据
    df['总数'] = cut_group_all
    df['坏样本'] = cut_y
    df['好样本'] = cut_n
    
    # 3、统计坏样本率和好样本率
    df['坏样本%'] = df['坏样本'] / df['坏样本'].sum()
    df['好样本%'] = df['好样本'] / df['好样本'].sum()
    
    # 4、计算WOE值
    df['WOE'] = np.log(df['坏样本%']/ df['好样本%'])
    df = df.replace({'WOE': {np.inf: 0, -np.inf: 0}})
    
    # 5.计算各个分箱的IV值
    df['IV'] = df['WOE'] * (df['坏样本%'] - df['好样本%'])

    # 6.汇总各个分箱的IV值，获得特征变量的IV值
    iv = df['IV'].sum()
    
    print(iv)

In [2]:
# 读取数据
data = pd.read_excel('datasets/股票客户流失.xlsx')
data.head()

,账户资金（元）,最后一次交易距今时间（天）,上月交易佣金（元）,本券商使用时长（年）,是否流失
0,22686.5,297,149.25,0,0
1,190055.0,42,284.75,2,0
2,29733.5,233,269.25,0,1
3,185667.5,44,211.50,3,0
4,33648.5,213,353.50,0,1


In [11]:
# 利用创建好的自定义函数计算第1个特征变量“账户资金（元）”的IV值
cal_iv(data, 4, '账户资金（元）', '是否流失')

0.15205722409339645


In [12]:
# 通过for循环计算出所有特征变量的IV值
for i in data.columns[:-1]:  # data.columns用于获取所有列名，因为我们考量的是特征变量，不需要最后一列目标变量“是否流失”，所以写成data.columns[:-1]
    print(i + '的IV值为：')
    cal_iv(data, 4, i, '是否流失') # 调用函数

账户资金（元）的IV值为：
0.15205722409339645
最后一次交易距今时间（天）的IV值为：
0.2508468300174099
上月交易佣金（元）的IV值为：
0.30811632146662304
本券商使用时长（年）的IV值为：
0.6144219248359752


上述IV值从高到低排序，结果为：本券商使用时长（年）＞上月交易佣金（元）＞最后一次交易距今时间（天）＞账户资金（元）。可得出结论：“本券商使用时长（年）”的信息量最大，而“账户资金（元）”的信息量最小，预测能力最低。这其实也是搭建逻辑回归模型时判断特征重要性的一个方式。